In [12]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from sklearn.metrics import precision_score, recall_score
%matplotlib inline

## Analysing data

In [13]:
data, labels = [], []

label_map = {
    "Iris-setosa": 0, 
    "Iris-versicolor": 1, 
    "Iris-virginica": 2
}

with open(os.path.join('iris','iris.data'),'r') as f:
    for row in f:
        if len(row.strip())>0:
            tokens = row.split(',')
            data.append(tokens[0:4])
            labels.append(label_map[tokens[4].strip()])
        
print("Sample data")
print(data[:5])
print(labels[:5])


# Convert to numpy array
data = np.array(data)
labels = np.array(labels)

onehot_labels = np.zeros(shape=(data.shape[0],3))
onehot_labels[np.arange(0,data.shape[0]),labels]=1.0

print("Shapes of datasets")
print(data.shape)
print(labels.shape)
print(onehot_labels.shape)

Sample data
[['5.1', '3.5', '1.4', '0.2'], ['4.9', '3.0', '1.4', '0.2'], ['4.7', '3.2', '1.3', '0.2'], ['4.6', '3.1', '1.5', '0.2'], ['5.0', '3.6', '1.4', '0.2']]
[0, 0, 0, 0, 0]
Shapes of datasets
(150, 4)
(150,)
(150, 3)


## Defining a graph for fully-connected network

In [37]:
tf.reset_default_graph()

batch_size = 25
# Define an input
tf_in = tf.placeholder(
    name='input', shape=[batch_size,4], dtype=tf.float32
)

# 1st output layer
tf_fulcon1_w = tf.get_variable(
    'layer1_w', shape=[4,100], 
    initializer=tf.glorot_uniform_initializer)
tf_fulcon1_b = tf.get_variable(
    'layer1_b', shape=[100], 
    initializer=tf.glorot_uniform_initializer
)

# computing output
tf_out1 = tf.matmul(tf_in, tf_fulcon1_w) + tf_fulcon1_b
tf_out1 = tf.nn.relu(tf_out1)

# Second layer
tf_fulcon2_w = tf.get_variable(
    'layer2_w', shape=[100,3], 
    initializer=tf.glorot_uniform_initializer
)
tf_fulcon2_b = tf.get_variable(
    "layer2_b", shape=[3],
    initializer=tf.glorot_uniform_initializer
)

# Compute 2nd output (Score of the neural network)
tf_out = tf.nn.relu(
    tf.matmul(tf_out1, tf_fulcon2_w)+tf_fulcon2_b)

# Softmax
tf_pred = tf.nn.softmax(tf_out)

## Defining loss and optimizer

In [31]:
tf_label = tf.placeholder(
    name='labels', shape=[batch_size, 3], dtype=tf.float32)

tf_loss = tf.nn.softmax_cross_entropy_with_logits_v2(
    labels=tf_label, logits=tf_out)

tf_opt = tf.train.MomentumOptimizer(
    learning_rate=1e-4, momentum=0.9
).minimize(tf_loss)

## Splitting the dataset to train and test

In [32]:
np.random.seed(100)

data_mask = np.random.choice(
    [True,False], size=150, p=[0.8, 0.2]
)

train_data = data[data_mask,:]
train_labels = labels[data_mask]
train_onehot_labels = onehot_labels[data_mask, :]

test_data = data[~data_mask, :]
test_labels = labels[~data_mask]
test_onehot_labels = onehot_labels[~data_mask, :]

print('Dataset sizes')
print(train_data.shape)
print(test_data.shape)

#print(train_data)
print(train_data[:5,:])

Dataset sizes
(119, 4)
(31, 4)
[['5.1' '3.5' '1.4' '0.2']
 ['4.9' '3.0' '1.4' '0.2']
 ['4.7' '3.2' '1.3' '0.2']
 ['5.0' '3.6' '1.4' '0.2']
 ['5.4' '3.9' '1.7' '0.4']]


## Running the neural network

In [33]:
def compute_accuracy(labels, predictions):
    
    true_classes = np.argmax(labels, axis=1).ravel()
    pred_classes = np.argmax(predictions, axis=1).ravel()
    
    accuracy = np.sum((true_classes==pred_classes))/true_classes.shape
    return accuracy

In [35]:
with tf.Session() as sess:
    
    # Initialize all global variables
    # You can also use: sess.run(tf.global_variables_initializer())
    tf.global_variables_initializer().run()
    
    for epoch in range(100):
        losses = []
        accuracy = []
        
        """ Train phase """
        # Iterate training dataset
        for bi in range(0, train_data.shape[0], batch_size):
            
            # Getting a random slice of data to train the network
            rand_idx = np.random.randint(
                0,train_data.shape[0],size=batch_size)
            b_data = train_data[rand_idx, :]
            b_labels = train_onehot_labels[rand_idx, :]

            pred, loss, _ = sess.run([tf_pred, tf_loss, tf_opt], 
                                     feed_dict={
                                         tf_in:b_data,
                                         tf_label:b_labels
                                     })
            losses.append(loss)
            accuracy.append(compute_accuracy(b_labels, pred))
        print(
            "Mean loss for epoch {}: {}".format(
                epoch, np.mean(losses)))
        print("\tTrain accuracy: {}".format(np.mean(accuracy)))
        
        """ Testing phase """
        test_accuracy=[]
        for bi in range(0, test_data.shape[0], batch_size):
            b_data = test_data[bi:bi+batch_size, :]
            b_labels = test_onehot_labels[bi:bi+batch_size, :]
                
            pred = sess.run(tf_pred, feed_dict={tf_in:b_data})
            test_accuracy.append(
                compute_accuracy(b_labels, pred)
            )
            print(test_accuracy)
        print("\tTest accuracy: {}".format(
            np.mean(test_accuracy)))

Mean loss for epoch 0: 1.1713685989379883
	Train accuracy: 0.4
[array([0.2])]
[array([0.2]), array([1.])]
	Test accuracy: 0.6
Mean loss for epoch 1: 1.1014598608016968
	Train accuracy: 0.328
[array([0.2])]
[array([0.2]), array([1.])]
	Test accuracy: 0.6
Mean loss for epoch 2: 0.9917137026786804
	Train accuracy: 0.4720000000000001
[array([0.8])]
[array([0.8]), array([0.])]
	Test accuracy: 0.4
Mean loss for epoch 3: 0.8078797459602356
	Train accuracy: 0.6559999999999999
[array([0.6])]
[array([0.6]), array([0.33333333])]
	Test accuracy: 0.4666666666666667
Mean loss for epoch 4: 0.7524250149726868
	Train accuracy: 0.696
[array([0.68])]
[array([0.68]), array([1.])]
	Test accuracy: 0.8400000000000001
Mean loss for epoch 5: 0.7009680867195129
	Train accuracy: 0.6960000000000001
[array([0.88])]
[array([0.88]), array([0.5])]
	Test accuracy: 0.69
Mean loss for epoch 6: 0.6203170418739319
	Train accuracy: 0.8
[array([0.92])]
[array([0.92]), array([0.83333333])]
	Test accuracy: 0.8766666666666667


[array([0.96])]
[array([0.96]), array([1.])]
	Test accuracy: 0.98
Mean loss for epoch 68: 0.17308852076530457
	Train accuracy: 0.96
[array([0.96])]
[array([0.96]), array([1.])]
	Test accuracy: 0.98
Mean loss for epoch 69: 0.15963977575302124
	Train accuracy: 1.0
[array([0.96])]
[array([0.96]), array([1.])]
	Test accuracy: 0.98
Mean loss for epoch 70: 0.16842344403266907
	Train accuracy: 0.992
[array([0.96])]
[array([0.96]), array([1.])]
	Test accuracy: 0.98
Mean loss for epoch 71: 0.16428402066230774
	Train accuracy: 0.992
[array([0.96])]
[array([0.96]), array([1.])]
	Test accuracy: 0.98
Mean loss for epoch 72: 0.18369534611701965
	Train accuracy: 0.992
[array([0.96])]
[array([0.96]), array([1.])]
	Test accuracy: 0.98
Mean loss for epoch 73: 0.17121422290802002
	Train accuracy: 0.984
[array([0.96])]
[array([0.96]), array([1.])]
	Test accuracy: 0.98
Mean loss for epoch 74: 0.1696244180202484
	Train accuracy: 0.976
[array([0.96])]
[array([0.96]), array([1.])]
	Test accuracy: 0.98
Mean lo

c:\anaconda3\envs\tekbac.deeplearning\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Loss for epoch 0 is: 1.2853949069976807
Test accuracy: 0.32
Test precision: 0.1024
Test recall: 0.32
Loss for epoch 1 is: 1.250168800354004
Test accuracy: 0.32
Test precision: 0.1024
Test recall: 0.32
Loss for epoch 2 is: 1.201775074005127
Test accuracy: 0.32
Test precision: 0.1024
Test recall: 0.32
Loss for epoch 3 is: 1.1585885286331177
Test accuracy: 0.32
Test precision: 0.1024
Test recall: 0.32
Loss for epoch 4 is: 1.1284375190734863
Test accuracy: 0.32
Test precision: 0.1024
Test recall: 0.32
Loss for epoch 5 is: 1.110302448272705
Test accuracy: 0.32
Test precision: 0.1024
Test recall: 0.32
Loss for epoch 6 is: 1.09916353225708
Test accuracy: 0.32
Test precision: 0.1024
Test recall: 0.32
Loss for epoch 7 is: 1.0902526378631592
Test accuracy: 0.32
Test precision: 0.1024
Test recall: 0.32
Loss for epoch 8 is: 1.081242561340332
Test accuracy: 0.32
Test precision: 0.1024
Test recall: 0.32
Loss for epoch 9 is: 1.0717322826385498
Test accuracy: 0.32
Test precision: 0.1024
Test recall: 0

Loss for epoch 90 is: 0.8924829363822937
Test accuracy: 0.6
Test precision: 0.6222222222222222
Test recall: 0.6
Loss for epoch 91 is: 0.8916943073272705
Test accuracy: 0.6
Test precision: 0.6222222222222222
Test recall: 0.6
Loss for epoch 92 is: 0.8909199237823486
Test accuracy: 0.6
Test precision: 0.6222222222222222
Test recall: 0.6
Loss for epoch 93 is: 0.8901498317718506
Test accuracy: 0.6
Test precision: 0.6222222222222222
Test recall: 0.6
Loss for epoch 94 is: 0.8893840909004211
Test accuracy: 0.6
Test precision: 0.6222222222222222
Test recall: 0.6
Loss for epoch 95 is: 0.8886203765869141
Test accuracy: 0.6
Test precision: 0.6222222222222222
Test recall: 0.6
Loss for epoch 96 is: 0.8878573775291443
Test accuracy: 0.6
Test precision: 0.6222222222222222
Test recall: 0.6
Loss for epoch 97 is: 0.8870989084243774
Test accuracy: 0.6
Test precision: 0.6222222222222222
Test recall: 0.6
Loss for epoch 98 is: 0.8863464593887329
Test accuracy: 0.6
Test precision: 0.6222222222222222
Test reca

In [38]:
print(tf_in.shape)

(25, 4)
